In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import Dataset
from bitsandbytes.nn import Linear8bitLt
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model

c:\Users\frane\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train_df = pd.read_csv('data/hate_train.csv')
# with open('data/hate_test_data.txt', 'r', encoding='utf-8') as file:
#     test_sentences = file.read().splitlines()
#
# test_df = pd.DataFrame({'sentence': test_sentences})
train_df = pd.read_csv('data/hate_train_cleaned.csv')
test_df = pd.read_csv('data/hate_test_cleaned.csv')

In [3]:
train_df.head()

,sentence,label
0,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
1,Brawo ty Daria kibic ma być na dobre i złe,0
2,"Super, polski premier składa kwiaty na grobach...",0
3,Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0


In [4]:
test_df.head()

,sentence
0,"Spoko, jak im Duda z Morawieckim zamówią po pi..."
1,"Ale on tu nie miał szans jej zagrania, a ta 'p..."
2,"No czy Prezes nie miał racji, mówiąc,ze to są ..."
3,Przecież to nawet nie jest przewrotka 😂
4,Owszem podatki tak. Ale nie w takich okoliczno...


In [5]:
# train_df_cleaned = train_df.copy()
# test_df_cleaned = test_df.copy()

def remove_anonymized_account(text):
    if isinstance(text, str):
        return text.replace('@anonymized_account ', '')
    return text

In [6]:
train_df = train_df[~train_df['sentence'].str.startswith('RT ', na=False)]
print(f"Liczba próbek po usunięciu retweetów: {len(train_df)}")

Liczba próbek po usunięciu retweetów: 9387


In [7]:
# train_df_cleaned['sentence'] = train_df_cleaned['sentence'].apply(remove_anonymized_account)
# test_df_cleaned['sentence'] = test_df_cleaned['sentence'].apply(remove_anonymized_account)

In [8]:
# train_df_cleaned.to_csv('data/hate_train_cleaned.csv', index=False)
# test_df_cleaned.to_csv('data/hate_test_cleaned.csv', index=False)

In [9]:
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['label'])

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_df)

Konfiguracja modelu i tokenizera

In [10]:
model_name = "szymonrucinski/Curie-7B-v1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 23152.48 examples/s]


Konfiguracja LoRA dla efektywnego fine-tuningu

In [11]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

In [12]:
model_path = "szymonrucinski/Curie-7B-v1"

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)

model.config.pad_token_id = tokenizer.pad_token_id
model = get_peft_model(model, peft_config)

Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.69s/it]
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at szymonrucinski/Curie-7B-v1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.69s/it]
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at szymonrucinski/Curie-7B-v1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./models/curie-hate-speech-experimental",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    push_to_hub=False,
    bf16=True,
)

Definiowanie metryki do ewaluacji

In [14]:
def compute_metrics(eval_preds):
    import numpy as np
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    
    positive_mask = labels == 1
    positive_predictions = predictions[positive_mask]
    positive_labels = labels[positive_mask]
    positive_accuracy = accuracy_score(positive_labels, positive_predictions) if len(positive_labels) > 0 else 0.0
    
    negative_mask = labels == 0
    negative_predictions = predictions[negative_mask]
    negative_labels = labels[negative_mask]
    negative_accuracy = accuracy_score(negative_labels, negative_predictions) if len(negative_labels) > 0 else 0.0
    
    precision_weight = 0.7
    recall_weight = 0.2
    f1_weight = 0.1
    weighted_score = (precision * precision_weight + recall * recall_weight + f1 * f1_weight)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'weighted_precision_score': weighted_score,
        'positive_accuracy': positive_accuracy,
        'negative_accuracy': negative_accuracy,
    }

Inicjalizowanie Trainera i rozpoczęcie treningu

In [15]:
print("Rozpoczynanie treningu...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

trainer.train()#resume_from_checkpoint="./models/curie-hate-speech/checkpoint-2112")

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Rozpoczynanie treningu...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Weighted Precision Score,Positive Accuracy,Negative Accuracy
1,0.309800,0.405809,0.920128,0.571429,0.052632,0.096386,0.420165,0.052632,0.996524
2,0.246400,0.328377,0.915868,0.473684,0.355263,0.406015,0.443233,0.355263,0.965238
3,0.150600,0.394333,0.915868,0.466667,0.276316,0.347107,0.416641,0.276316,0.972190


TrainOutput(global_step=3168, training_loss=0.2738236628397547, metrics={'train_runtime': 11623.2104, 'train_samples_per_second': 2.18, 'train_steps_per_second': 0.273, 'total_flos': 1.3638301348213555e+17, 'train_loss': 0.2738236628397547, 'epoch': 3.0})

Zapisanie modelu i ewaluacja danych

In [16]:
model.save_pretrained("./models/curie-hate-speech-final")
tokenizer.save_pretrained("./models/curie-hate-speech-final")

('./models/curie-hate-speech-final\\tokenizer_config.json',
 './models/curie-hate-speech-final\\special_tokens_map.json',
 './models/curie-hate-speech-final\\tokenizer.json')

In [17]:
print("Ewaluacja modelu...")
eval_results = trainer.evaluate()
print(f"Wyniki ewaluacji: {eval_results}")

Ewaluacja modelu...


Wyniki ewaluacji: {'eval_loss': 0.3283769488334656, 'eval_accuracy': 0.9158679446219382, 'eval_precision': 0.47368421052631576, 'eval_recall': 0.35526315789473684, 'eval_f1': 0.40601503759398494, 'eval_weighted_precision_score': 0.44323308270676687, 'eval_positive_accuracy': 0.35526315789473684, 'eval_negative_accuracy': 0.9652375434530707, 'eval_runtime': 74.2988, 'eval_samples_per_second': 12.638, 'eval_steps_per_second': 3.163, 'epoch': 3.0}


In [18]:
raw_predictions = trainer.predict(test_dataset)
predictions = raw_predictions.predictions.argmax(axis=1)

test_df_with_predictions = test_df.copy()

output_path = 'pred.csv'
output_df = pd.DataFrame({'prediction': predictions})
output_df.to_csv(output_path, index=False, header=False)

print(output_df.head(5))

hate_speech_count = sum(predictions)
hate_speech_percentage = (hate_speech_count / len(predictions)) * 100
print(f"\nLiczba tekstów sklasyfikowanych jako mowa nienawiści: {hate_speech_count} ({hate_speech_percentage:.2f}%)")


   prediction
0           0
1           0
2           0
3           0
4           0

Liczba tekstów sklasyfikowanych jako mowa nienawiści: 68 (6.80%)
